In [ ]:
import pypsa
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
n=pypsa.Network()

#Snapshots
n.set_snapshots(value=pd.date_range(freq="h", start="2013-01-01", end="2014-01-01", inclusive='left'))
n.snapshot_weightings[:] *= 8760.0 / n.snapshot_weightings.sum()
Nyears = n.snapshot_weightings.objective.sum() / 8760.0

# Load yaml files
import yaml

with open(r'config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
 
        print(config)

In [2]:
buses = 500

for i in range(buses):
    n.add("Bus", [i], v_nom=20.0)

n.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
[0],20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
[1],20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
[2],20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
[3],20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
[4],20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
...,...,...,...,...,...,...,...,...,...,...,...
[495],20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
[496],20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
[497],20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,


In [ ]:
for i in range(buses):
    n.madd(
        "Line",
        [i],
        bus0="My bus {}".format(i),
        bus1="My bus {}".format((i + 1) % buses),
        x=0.1,
        r=0.01,
    )

In [4]:
n.lines

,bus0,bus1,x,r,type,g,b,s_nom,s_nom_extendable,s_nom_min,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
0,My bus 0,My bus 1,0.1,0.01,,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,My bus 1,My bus 2,0.1,0.01,,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,My bus 2,My bus 3,0.1,0.01,,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,My bus 3,My bus 4,0.1,0.01,,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,My bus 4,My bus 5,0.1,0.01,,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,My bus 495,My bus 496,0.1,0.01,,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,My bus 496,My bus 497,0.1,0.01,,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,My bus 497,My bus 498,0.1,0.01,,0.0,0.0,0.0,False,0.0,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

In [6]:
# technologies_path={
#     'onwind': "renewable_profiles/profile_onwind.nc",
#     'solar': "renewable_profiles/profile_solar.nc"
# }

In [7]:
tech_costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

idx = pd.IndexSlice

def calculate_annuity(n, r):
    """
    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    """
    if isinstance(r, pd.Series):
        return pd.Series(1 / n, index=r.index).where(
            r == 0, r / (1.0 - 1.0 / (1.0 + r) ** n)
        )
    elif r > 0:
        return r / (1.0 - 1.0 / (1.0 + r) ** n)
    else:
        return 1 / n

In [8]:
tech_costs='C://Users//denis//OneDrive//Desktop//Mini grids//pypsa-distribution//costs.csv'

In [9]:
def load_costs(tech_costs, config, elec_config, Nyears=1):
    """
    set all asset costs and other parameters
    """
    costs = pd.read_csv(tech_costs, index_col=list(range(3))).sort_index()

    # correct units to MW and EUR
    costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
    costs.loc[costs.unit.str.contains("USD"), "value"] *= config["USD2013_to_EUR2013"]

    costs = (
        costs.loc[idx[:, config["year"], :], "value"]
        .unstack(level=2)
        .groupby("technology")
        .sum(min_count=1)
    )

    costs = costs.fillna(
        {
            "CO2 intensity": 0,
            "FOM": 0,
            "VOM": 0,
            "discount rate": config["discountrate"],
            "efficiency": 1,
            "fuel": 0,
            "investment": 0,
            "lifetime": 25,
        }
    )

    costs["capital_cost"] = (
        (
            calculate_annuity(costs["lifetime"], costs["discount rate"])
            + costs["FOM"] / 100.0
        )
        * costs["investment"]
        * Nyears
    )

    costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
    costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]

    costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

    costs = costs.rename(columns={"CO2 intensity": "co2_emissions"})

    costs.at["OCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]
    costs.at["CCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]

    costs.at["solar", "capital_cost"] = 0.5 * (
        costs.at["solar-rooftop", "capital_cost"]
        + costs.at["solar-utility", "capital_cost"]
    )

    def costs_for_storage(store, link1, link2=None, max_hours=1.0):
        capital_cost = link1["capital_cost"] + max_hours * store["capital_cost"]
        if link2 is not None:
            capital_cost += link2["capital_cost"]
        return pd.Series(
            dict(capital_cost=capital_cost, marginal_cost=0.0, co2_emissions=0.0)
        )

    max_hours = elec_config["max_hours"]
    costs.loc["battery"] = costs_for_storage(
        costs.loc["battery storage"],
        costs.loc["battery inverter"],
        max_hours=max_hours["battery"],
    )
    costs.loc["H2"] = costs_for_storage(
        costs.loc["hydrogen storage"],
        costs.loc["fuel cell"],
        costs.loc["electrolysis"],
        max_hours=max_hours["H2"],
    )

    for attr in ("marginal_cost", "capital_cost"):
        overwrites = config.get(attr)
        if overwrites is not None:
            overwrites = pd.Series(overwrites)
            costs.loc[overwrites.index, attr] = overwrites

    return costs

In [10]:
costs = load_costs(
    tech_costs,
    config["costs"],
    config["electricity"],
    Nyears,
    )

In [11]:

def attach_wind_and_solar(n, costs, technologies, extendable_carriers):
    
    for tech in technologies:
        
        # with xr.open_dataset(technologies_path[tech]) as ds:
        with xr.open_dataset(f"renewable_profiles/profile_{tech}.nc") as ds:

            if ds.indexes["bus"].empty:
                continue   

            suptech = tech.split("-", 2)[0]


            n.madd(
            "Generator",
            ds.indexes["bus"],
            " " + tech,
            bus=ds.indexes["bus"],
            carrier=tech,
            p_nom_extendable=tech in extendable_carriers["Generator"],
            p_nom_max=ds["p_nom_max"].to_pandas(),
            weight=ds["weight"].to_pandas(),
            marginal_cost=costs.at[suptech, "marginal_cost"],
            capital_cost=costs.at[tech, "capital_cost"],
            efficiency=costs.at[suptech, "efficiency"],
            p_max_pu=ds["profile"].transpose("time", "bus").to_pandas(),
            )


In [ ]:
attach_wind_and_solar(
    n,
    costs,
    config["renewable"],
    config["electricity"]["extendable_carriers"],
    )

In [ ]:
n.generators

In [64]:
#I create the load file
import numpy as np
p_set=np.random.rand(len(n.snapshots),70)
p_set=p_set*100
df=pd.DataFrame(p_set)
load = df.set_index([n.snapshots]) #.rename_axis("time")
old_columns = list(df)

a=["bus 0", "bus 1", "bus 2", "bus 3", "bus 4", "bus 5", "bus 6", "bus 7", "bus 8", "bus 9", 
   "bus 10", "bus 11", "bus 12", "bus 13", "bus 14", "bus 15", "bus 16", "bus 17", "bus 18", "bus 19", 
   "bus 20", "bus 21", "bus 22", "bus 23", "bus 24", "bus 25", "bus 26", "bus 27", "bus 28", "bus 29", 
   "bus 30", "bus 31", "bus 32", "bus 33", "bus 34", "bus 35", "bus 36", "bus 37", "bus 38", "bus 39", 
   "bus 40", "bus 41", "bus 42", "bus 43", "bus 44", "bus 45", "bus 46", "bus 47", "bus 48", "bus 49", 
   "bus 50", "bus 51", "bus 52", "bus 53", "bus 54", "bus 55", "bus 56", "bus 57", "bus 58", "bus 59",
   "bus 60", "bus 61", "bus 62", "bus 63", "bus 64", "bus 65", "bus 66", "bus 67", "bus 68", "bus 69"]
   
new_col_list=a

# rename the columns:
load=load.rename(columns={old_columns[idx]: name for  (idx, name) in enumerate(new_col_list)})

In [ ]:
df.head()

In [66]:
def attach_load(n, load_paths):
    
    for i in range(70):
     
        n.madd("Load", [f"load {i}"], bus=[f"bus {i}"] , carrier="AC", p_set=load)
      
        

In [ ]:
attach_load(n, load)

In [59]:
n.loads

,bus,carrier,type,p_set,q_set,sign
Load,,,,,,
0,bus 0,AC,,0.0,0.0,-1.0
1,bus 1,AC,,0.0,0.0,-1.0
2,bus 2,AC,,0.0,0.0,-1.0
3,bus 3,AC,,0.0,0.0,-1.0
4,bus 4,AC,,0.0,0.0,-1.0
...,...,...,...,...,...,...
load 65,bus 65,AC,,0.0,0.0,-1.0
load 66,bus 66,AC,,0.0,0.0,-1.0
load 67,bus 67,AC,,0.0,0.0,-1.0


In [ ]:
print(n)

PyPSA Network
Components:
 - Bus: 500
 - Generator: 449
 - Line: 500
 - Load: 70
Snapshots: 8760
